In [ ]:
import os, sys
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import datetime
import pickle
from lightgbm import *
from sklearn.model_selection import *
from sklearn.ensemble import *
import scipy
from xgboost import *
from catboost import *

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%d/%m/%y')

In [ ]:
trainDF = pd.read_csv('dataset/train.csv', parse_dates=['booking_date', 'checkin_date', 'checkout_date'], date_parser=dateparse)
print('Done loading training data')
testDF = pd.read_csv('dataset/test.csv', parse_dates=['booking_date', 'checkin_date', 'checkout_date'], date_parser=dateparse)
print('Done loading testing data')

In [ ]:
cf = [
 'channel_code',
 'main_product_code',
 'resort_region_code',
 'resort_type_code',
 'room_type_booked_code',
 'season_holidayed_code',
 'state_code_residence',
 'state_code_resort',
 'booking_type_code',
 'xxx_ignore_pax_or_ch_ad',
 'xxx_persontravellingid',
 'xxx_member_age_buckets',
 'xxx_cluster',
 'xxx_reservationstatusid_code',
 'xxx_resort_code']

nf = [
 'numberofadults',
 'numberofchildren',
 'roomnights',
 'total_pax',
 'xxx_checkin_boarding_diff',
 'xxx_checkout_checkin_diff',
 'xxx_new_checkin',
 'xxx_stayed_minus_roomnights',
 'xxx_xcheckin_nhf',
 'xxx_diff_people',
 'xxx_child_plus_adults',
 'xxx_child_fraction',
 'xxx_adult_fraction']

In [ ]:
newcf = []
for feature in cf:
    allValues = set(trainDF[feature])
    for value in allValues:
        newcf.append(feature+"_"+str(value))
        
print len(newcf)

In [ ]:
usfDict = {}

users = list(set(trainDF['memberid']))+list(set(testDF['memberid']))

for user in users:
    usfDict[user] = {}
    for f in newcf:
        usfDict[user][f] = 0

In [ ]:
userCount = {}
for i in range(len(trainDF)):
    userCount[trainDF.loc[i, 'memberid']]=trainDF.loc[i, 'xxx_member_feature']
    
for i in range(len(testDF)):
    userCount[testDF.loc[i, 'memberid']]=testDF.loc[i, 'xxx_member_feature']

In [ ]:
for i in range(len(trainDF)):
    mid = trainDF.loc[i, 'memberid']
    for f in cf:
        fv = trainDF.loc[i, f]
        fv = f+'_'+str(fv)
        usfDict[mid][fv]+=1
    if i%1000==0:
        print(i)

In [ ]:
for i in range(len(testDF)):
    mid = testDF.loc[i, 'memberid']
    for f in cf:
        fv = testDF.loc[i, f]
        fv = f+'_'+str(fv)
        usfDict[mid][fv]+=1

In [ ]:
for user in usfDict.keys():
    for fv in usfDict[user].keys():
        usfDict[user][fv] = float(usfDict[user][fv])/userCount[user]

In [ ]:
nusfDict = {}

for user in users:
    nusfDict[user] = {}
    for f in nf:
        nusfDict[user][f] = 0

In [ ]:
for i in range(len(trainDF)):
    mid = trainDF.loc[i, 'memberid']
    for f in nf:
        nusfDict[mid][f]+=trainDF.loc[i, f]

In [ ]:
for i in range(len(testDF)):
    mid = testDF.loc[i, 'memberid']
    for f in nf:
        nusfDict[mid][f]+=testDF.loc[i, f]

In [ ]:
for user in nusfDict.keys():
    for fv in nusfDict[user].keys():
        nusfDict[user][fv] = float(nusfDict[user][fv])/userCount[user]

In [ ]:
newSetTrain = {}
newSetTest = {}

for f in nf:
    newSetTrain[f] = []
    newSetTest[f] = []
for f in newcf:
    newSetTrain[f] = []
    newSetTest[f] = []

In [ ]:
for i in range(len(trainDF)):
    mid = trainDF.loc[i, 'memberid']
    for f in nusfDict[mid].keys():
        newSetTrain[f].append(nusfDict[mid][f])
    for f in usfDict[mid].keys():
        newSetTrain[f].append(usfDict[mid][f])

In [ ]:
for i in range(len(testDF)):
    mid = testDF.loc[i, 'memberid']
    for f in nusfDict[mid].keys():
        newSetTest[f].append(nusfDict[mid][f])
    for f in usfDict[mid].keys():
        newSetTest[f].append(usfDict[mid][f])

In [ ]:
newDatasetTrain = {}
newDatasetTest = {}

for f in newSetTrain.keys():
    newDatasetTrain['yyy_'+f] = newSetTrain[f]
    
for f in newSetTest.keys():
    newDatasetTest['yyy_'+f] = newSetTest[f]

In [ ]:
newTrainDF = trainDF.copy()
newTestDF = testDF.copy()

In [ ]:
for f in newDatasetTrain.keys():
    newTrainDF[f] = newDatasetTrain[f]
    
for f in newDatasetTest.keys():
    newTestDF[f] = newDatasetTest[f]

In [ ]:
testIds = np.array(newTestDF['reservation_id'].values)
y = np.array(newTrainDF['amount_spent_per_room_night_scaled'].values)
groups = np.array(newTrainDF['memberid'].values)

In [ ]:
copyTrain = newTrainDF.copy()
copyTest = newTestDF.copy()

In [ ]:
del copyTrain['reservation_id']
del copyTrain['amount_spent_per_room_night_scaled']
del copyTest['reservation_id']
del copyTrain['memberid']
del copyTest['memberid']

In [ ]:
del trainDF['reservation_id']
del trainDF['amount_spent_per_room_night_scaled']
del testDF['reservation_id']
del trainDF['memberid']
del testDF['memberid']

In [ ]:
X = np.array(copyTrain.values)
XTest = np.array(copyTest.values)
featureColumns = np.array(copyTrain.columns)

In [ ]:
X1 = np.array(trainDF.values)
XTest1 = np.array(testDF.values)
featureColumns1 = np.array(trainDF.columns)

In [ ]:
pickle.dump([X, y, XTest, testIds, featureColumns, groups], open('freshPickles/cleanExperimentDataF=226.pk', 'wb'))